In [4]:
import numpy as np
import pandas as pd
import torch
from transformers import *
from tqdm import tqdm
import os
from transformers import BartTokenizer
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

video_caption_dict = np.load('/home1/kmayya/Pipeline/Pipeline/video_caption_dict.npy',allow_pickle='TRUE').item()

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model = model.to(device)

model.load_state_dict(torch.load("/home1/kmayya/Pipeline/t5_on_synthetic.pth"))
summary_dict = {} 

for i, (key, val) in enumerate(video_caption_dict.items()):
    if i >= 1000:
        break
    if key == '.ipynb_checkpoints':
        continue 
    print(f"video {i} = ", key) 
    document = " ".join(val) 

    inputs = tokenizer(f"summarize keyframe captions: + ${document}", return_tensors="pt")
    inputs.to(device)
    # Generate the output
    outputs = model.generate(
        **inputs, 
        max_length=1500,  # Maximum output length
        num_beams=5,    # Beam search for better results
        early_stopping=True
    )

    final_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("Generated Paragraph:", final_summary)
    summary_dict[key] = final_summary

loading file spiece.model from cache at /home1/kmayya/.cache/huggingface/hub/models--t5-base/snapshots/a9723ea7f1b39c1eae772870f3b547bf6ef7e6c1/spiece.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading file tokenizer.json from cache at /home1/kmayya/.cache/huggingface/hub/models--t5-base/snapshots/a9723ea7f1b39c1eae772870f3b547bf6ef7e6c1/tokenizer.json
loading configuration file config.json from cache at /home1/kmayya/.cache/huggingface/hub/models--t5-base/snapshots/a9723ea7f1b39c1eae772870f3b547bf6ef7e6c1/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  

video 0 =  dufQjqa3v6w
Generated Paragraph: a man in a green t-shirt and white pants appears to be in his late 40s or early 50s . he is wearing a green t-shirt and white pants and appears to be in his late 40s or early 50s . there are several framed pictures hanging on a wall in the background .
video 1 =  TEaugXCX1m4
Generated Paragraph: the image appears to be taken from a low angle, looking up at the surfer. The surfer is wearing a black wetsuit and is riding a white surfboard. He is wearing a black wetsuit and is riding a white surfboard. The water is a deep blue-green color and there are small waves visible in the background. The sky is clear and blue, and the overall mood of the image is peaceful and serene.
video 2 =  vvdmMyyAtN0
Generated Paragraph: the image is a close-up of a person's face and body. The person is wearing a black t-shirt and is holding a pair of scissors in their right hand. The scissors appear to be made of a durable material and are placed on a white tablecl

In [5]:
df = pd.read_csv('summarized_dense_descriptions_1000_final.csv')
df['t5_finetuned_on_synthetic_summarized_video_captions'] = df['video_id'].map(summary_dict)
df.dropna().to_csv('/home1/kmayya/Pipeline/summarized_dense_descriptions_1000_final.csv', index=False) 

In [6]:
df = pd.read_csv('summarized_dense_descriptions_1000_final.csv')
df

,video_id,description,youtube_url,dense_frame_descriptions,bart_summarized_video_captions,t5_finetuned_on_movie_summarized_video_captions,t5_finetuned_on_synthetic_summarized_video_captions
0,nwznKOuZM7w,Two people are seen moving around a kitchen qu...,https://www.youtube.com/watch?v=nwznKOuZM7w,"['a man in a kitchen, standing in front of a w...","The image is taken from a low angle, looking d...","+ $a man in a kitchen, standing in front of a ...","the image is taken from a low angle, looking d..."
1,ogQozSI5V8U,We see a hallway with a wooden floor. A dog i...,https://www.youtube.com/watch?v=ogQozSI5V8U,['a narrow hallway with a wooden floor. The ha...,"The image is taken from a low angle, looking d...",a dog standing on its hind legs in a hallway. ...,"the image is taken from a low angle, looking d..."
2,nHE7u40plD0,A woman and a man are sitting on the sidewalk ...,https://www.youtube.com/watch?v=nHE7u40plD0,"[""a group of people sitting on a brick patio. ...",The image shows a group of people gathered aro...,a group of people sitting on a brick patio. Th...,The image is a close-up of a person's face and...
3,69IsHpmRyfk,A young girl is seen sitting in a chair with a...,https://www.youtube.com/watch?v=69IsHpmRyfk,"[""The image is a close-up of a young girl's fa...",A person's hand is visible on the right side o...,a close-up of a young girl with blonde hair. S...,the image is a close-up of a young woman's fac...
4,D18b2IZpxk0,A woman is shown riding a camel past pyramids ...,https://www.youtube.com/watch?v=D18b2IZpxk0,['a person riding a camel in a desert-like lan...,The image is an abstract background with a gra...,a person riding a camel in a desert-like lands...,"the image is taken from a low angle, looking u..."
...,...,...,...,...,...,...,...
972,XftM9eALPy0,A large arena houses many people. Poles are s...,https://www.youtube.com/watch?v=XftM9eALPy0,['a gymnast performing on the uneven bars in a...,The image appears to have been taken from a hi...,a male gymnast performing on the uneven bars i...,the image appears to have been taken from a hi...
973,9Hw3xvJJTy0,Three wrestlers were in the ring and crowd isw...,https://www.youtube.com/watch?v=9Hw3xvJJTy0,['The image is a screenshot from a video game....,The image is a screenshot from a video game. I...,"The main focus of the image is on a wrestler, ...",the image is a screenshot from a video game. I...
974,pZZVzBD8bZY,Various shots are shown of large crowd celebra...,https://www.youtube.com/watch?v=pZZVzBD8bZY,['a close-up of the back of a white jersey wit...,A close-up of the back of a white jersey with ...,a close-up of the back of a white jersey with ...,the image appears to be taken from a low angle...
975,OCT2h1kmZoA,We see an animated opening sequence. We see m...,https://www.youtube.com/watch?v=OCT2h1kmZoA,['The image is a logo for a board shop called ...,"The image is a logo for a board shop called ""L...","a board shop called ""LFSNS Board Shop"" The bac...",the image is a logo for a snowboard shop calle...
